# Selecting Central Jets

To work we need a ServiceX instance to run against. For this demo we'll use the Open Data instance located at University of Nebraska.

All ServiceX instances require an account. To get an account go to [https://coffea-opendata.casa/](https://coffea-opendata.casa/), log in, and wait to be approved.

## Stream of Events (Source)

Most commonly called the dataset - it specifies the source we will be pulling from. 

For this simple demo, lets do a single file on an open link, at an `http` endpoint. This is the first `ttbar` file from the `analysis-grand-challenge` [master ntuple list](https://github.com/iris-hep/analysis-grand-challenge/blob/main/analyses/cms-open-data-ttbar/ntuples.json).

In [11]:
from func_adl_servicex import ServiceXSourceUpROOT

file ="https://xrootd-local.unl.edu:1094//store/user/AGC/datasets/RunIIFall15MiniAODv2/TT_TuneCUETP8M1_13TeV-powheg-pythia8/MINIAODSIM//PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1/00000/00DF0A73-17C2-E511-B086-E41D2D08DE30.root"

ds = ServiceXSourceUpROOT([file], treename="mini")

`ds`, short for dataset, is the stream of events source. Since this is a column wise ntuple, and `ds` gives us access to its contents.

In [14]:
data = (ds
    .Select(lambda e: {
        'pt': e['jet_pt'],
    })
    .AsAwkwardArray()
    .value()
)

ServerDisconnectedError: Server disconnected